# Problemas de prueba de hipótesis

## Ejercicio 1

Usted es un nutricionista que investiga dos tipos diferentes de dietas para ver si existe una diferencia significativa en la pérdida de peso después de un mes. Eliges dos grupos aleatorios de personas; un grupo sigue la primera dieta y el otro sigue la segunda. Al final del mes, se registra la pérdida de peso (en kg) de cada persona:

| Diet 1 | Diet 2 |
|:-------|:-------|
| 2.0 | 3.0 |
| 2.5 | 3.2 |
| 3.0 | 3.1 |
| 2.8 | 2.9 |
| 2.3 | 2.8 |
| 2.7 | 3.0 |
| 2.5 | 3.2 |

Con estos datos se busca responder a la siguiente pregunta: ¿Existe una diferencia significativa en la pérdida de peso promedio entre las personas que siguieron la primera dieta y las que siguieron la segunda dieta?

Para sacar conclusiones, siga los siguientes puntos:

-Enunciar la hipótesis: hipótesis nula y alternativa.
-Realizar la prueba para comprobar la hipótesis. Puede utilizar la prueba t de Student.
-Analizar las conclusiones.

In [1]:
import numpy as np
from scipy import stats

**Hipotesis**
- H₀: μ₁ = μ₂ (no hay diferencia en la media de pérdida de peso).
- H₁: μ₁ ≠ μ₂ (sí hay diferencia).


In [2]:
diet1 = np.array([2.0, 2.5, 3.0, 2.8, 2.3, 2.7, 2.5])
diet2 = np.array([3.0, 3.2, 3.1, 2.9, 2.8, 3.0, 3.2])

In [3]:
# 1. Supuestos: normalidad y homogeneidad de varianzas

n1 = stats.shapiro(diet1)
n2 = stats.shapiro(diet2)
lev = stats.levene(diet1, diet2)

print("Normalidad Diet1: W={:.3f}, p={:.3f}".format(n1.statistic, n1.pvalue))
print("Normalidad Diet2: W={:.3f}, p={:.3f}".format(n2.statistic, n2.pvalue))
print("Homogeneidad de varianzas: W={:.3f}, p={:.3f}".format(lev.statistic, lev.pvalue))

Normalidad Diet1: W=0.981, p=0.964
Normalidad Diet2: W=0.935, p=0.591
Homogeneidad de varianzas: W=2.271, p=0.158


In [4]:
# 2. t-test (usando equal_var según la homogeneidad de varianzas)

t_stat, p_val = stats.ttest_ind(diet1, diet2, equal_var=(lev.pvalue > 0.05))
print("t-statistic: {:.3f}".format(t_stat))
print("p-valor: {:.4f}".format(p_val))

t-statistic: -3.538
p-valor: 0.0041


**# 3. Interpretacion de p-value y medida de efecto**



*La probabilidad de observar una diferencia tan grande entre las medias por puro azar es muy baja.*

Por lo tanto, rechazamos la hipótesis nula (H₀).

Existe evidencia estadísticamente significativa de que las dos dietas generan pérdidas de peso distintas.

**La Dieta 2 produce una mayor pérdida de peso promedio que la Dieta 1.**

Para comprender si la diferencia es relevante en la práctica, calculamos Cohen’s d:

Un valor alrededor de d ≈ 2 (aprox.) indica un efecto muy grande.

Esto significa que la diferencia no solo es significativa, sino que además es muy notable en magnitud.

Conclusiones:

Hay una diferencia clara y contundente entre ambas dietas:

- La Dieta 2 supera consistentemente a la Dieta 1 en pérdida de peso.
- El resultado es estadísticamente significativo y de fuerte impacto práctico.

## ANOVA

**ANOVA** (*Analysis of Variance - Análisis de variación*) Es una técnica estadística utilizada para comparar las medidas de dos o más grupos. La idea detrás de ANOVA es descomponer la variabilidad total de los datos en dos componentes: variabilidad entre grupos y variabilidad dentro de los grupos:

- **Variabilidad entre grupos**: Esta variabilidad se refiere a las diferencias entre las medias del grupo. Si esta variabilidad es considerablemente mayor que la variabilidad dentro del grupo, podría ser una indicación de que al menos una de las medias del grupo es diferente.
- **Variabilidad dentro del grupo**: Esta variabilidad se refiere a la dispersión de los datos dentro de cada grupo. Si todos los grupos tienen una variabilidad similar, entonces cualquier diferencia notable en las medias de los grupos podría considerarse significativa.

Las hipótesis en ANOVA suelen incluir:

- **Hipótesis nula** ($H₀$): Las medias de todos los grupos son iguales.
- **Hipótesis alternativa** ($H₁$): Al menos una de las medias del grupo es diferente.

Si el resultado de la prueba ANOVA es significativo (por ejemplo, un valor p inferior a un umbral como 0,05), esto sugiere que al menos la media de un grupo es diferente.

## Ejercicio 2

Un agricultor decide probar tres tipos diferentes de fertilizantes para determinar si alguno es superior en términos de producción de maíz. El agricultor planta maíz en 15 parcelas idénticas y utiliza los tres fertilizantes (5 parcelas para cada tipo). Al final de la temporada mide el rendimiento de maíz (en kg) de cada parcela, con el siguiente resultado:

| Fertilizante 1 | Fertilizante 2 | Fertilizante 3 |
|:-------------|:-------------|:-------------|
| 20 | 22 | 24 |
| 21 | 21 | 23 |
| 20 | 23 | 22 |
| 19 | 22 | 23 |
| 20 | 21 | 24 |

Con estos datos busca responder la siguiente pregunta: ¿Existe una diferencia significativa en el rendimiento promedio del maíz entre los tres tipos de fertilizantes?

Para ayudarte, sigue los siguientes puntos:

-Enunciar la hipótesis: hipótesis nula y alternativa.
-Realizar la prueba ANOVA.
-Analizar las conclusiones.
-Si un fertilizante es mejor que otro, ¿cómo podemos saberlo?

In [5]:
# ANOVA + Tukey

import numpy as np
from scipy import stats
import pandas as pd
from statsmodels.stats.multicomp import pairwise_tukeyhsd

fert1 = [20, 21, 20, 19, 20]
fert2 = [22, 21, 23, 22, 21]
fert3 = [24, 23, 22, 23, 24]

# 1. Supuestos: normalidad por grupo y homogeneidad de varianzas

print("Normalidad Fert1:", stats.shapiro(fert1))
print("Normalidad Fert2:", stats.shapiro(fert2))
print("Normalidad Fert3:", stats.shapiro(fert3))
print("Homogeneidad de varianza (3 grupos):", stats.levene(fert1, fert2, fert3))

# 2. ANOVA one-way

f_stat, p_val = stats.f_oneway(fert1, fert2, fert3)
print("ANOVA: F = {:.3f}, p = {:.4f}".format(f_stat, p_val))


# 3. Post-hoc Tukey (si ANOVA significativa)

data = np.concatenate([fert1, fert2, fert3])
labels = ["F1"]*len(fert1) + ["F2"]*len(fert2) + ["F3"]*len(fert3)
tukey = pairwise_tukeyhsd(endog=data, groups=labels, alpha=0.05)
print("Resultados de Tukey HSD:")
print(tukey.summary())

# 4. Medida de efecto (Eta-squared)

grand_mean = np.mean(data)  # Media total

# Suma de cuadrados entre grupos

ss_between = (
    len(fert1) * (np.mean(fert1) - grand_mean)**2 +
    len(fert2) * (np.mean(fert2) - grand_mean)**2 +
    len(fert3) * (np.mean(fert3) - grand_mean)**2
)

# Suma total de cuadrados

ss_total = np.sum((data - grand_mean)**2)

# Eta squared

eta_squared = ss_between / ss_total

print("Eta squared (η²):", eta_squared)

Normalidad Fert1: ShapiroResult(statistic=np.float64(0.8834906936019853), pvalue=np.float64(0.32542950716414576))
Normalidad Fert2: ShapiroResult(statistic=np.float64(0.8810376385817489), pvalue=np.float64(0.3140395561440371))
Normalidad Fert3: ShapiroResult(statistic=np.float64(0.8810376385817489), pvalue=np.float64(0.3140395561440371))
Homogeneidad de varianza (3 grupos): LeveneResult(statistic=np.float64(0.22222222222222207), pvalue=np.float64(0.8039599174006208))
ANOVA: F = 20.316, p = 0.0001
Resultados de Tukey HSD:
Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj  lower  upper  reject
--------------------------------------------------
    F1     F2      1.8 0.0099 0.4572 3.1428   True
    F1     F3      3.2 0.0001 1.8572 4.5428   True
    F2     F3      1.4 0.0409 0.0572 2.7428   True
--------------------------------------------------
Eta squared (η²): 0.7719999999999995


**Resultados:**
- Shapiro por grupo: p > 0.05 → no violación de normalidad.
- Levene (3 grupos): p > 0.05 → homogeneidad de varianzas aceptable.
- ANOVA: F ≈ 20.316, p ≈ 0.0001 → p < 0.05, rechazamos H₀ (hay diferencias entre medias).
- Eta-squared ≈ 0.772 → efecto muy grande (≈77% de la variabilidad explicada por el tipo de fertilizante).

**Tukey HSD (post-hoc):**
- F1 vs F2: meandiff = 1.8, p-adj ≈ 0.0099 → diferencia significativa (F2 > F1).
- F1 vs F3: meandiff = 3.2, p-adj ≈ 0.0001 → diferencia significativa (F3 > F1).
- F2 vs F3: meandiff = 1.4, p-adj ≈ 0.0409 → diferencia significativa (F3 > F2).

**Conclusión práctica:** Fertilizante 3 produce el mayor rendimiento promedio, seguido por Fertilizante 2 y luego Fertilizante 1; las diferencias entre los tres son estadísticamente significativas bajo α = 0.05.